In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

Note: you may need to restart the kernel to use updated packages.


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

Initialize Vertex AI

In [4]:
PROJECT = !gcloud config get-value project
PROJECT_ID = PROJECT[0]
REGION = "us-west1"
vertexai.init(project=PROJECT_ID, location=REGION)

Load a generative model

In [5]:
model = GenerativeModel("gemini-pro")

Ideation<br>
means using the model to help you brainstorm new content: article titles, sections to include in a document, interview questions, etc.

In [6]:
creative_gen_config = GenerationConfig(temperature=1, top_p=0.8)

In [7]:
prompt = "What are some strategies for overcoming writer's block?"

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config) 
print(response.text)

## Strategies for Overcoming Writer's Block

Writer's block is a common challenge faced by writers of all levels. It can be frustrating and discouraging, but there are a number of strategies you can use to overcome it. Here are a few:

**1. Change your environment:** Sometimes a change of scenery can do wonders for creativity. Try writing in a different location, like a coffee shop, library, or park.
**2. Take a break:** If you've been staring at your screen for hours with no progress, it's time to step away. Take a walk, do some stretches, or listen to music. When you come back to your work, you'll feel refreshed and ready to start again.
**3. Freewrite:** This is a great way to get your thoughts flowing and to warm up your writing muscles. Just write whatever comes to mind, without worrying about grammar, punctuation, or making sense.
**4. Do some brainstorming:** If you're not sure where to start, try brainstorming some ideas. This could involve writing down a list of keywords, mind

In [8]:
prompt = "Provide ten interview questions for the role of prompt engineer."

response = model.generate_content(prompt,
                                  generation_config=creative_gen_config) 
print(response.text)

## Prompt Engineer Interview Questions:

**Technical Skills:**

1. **Explain the different prompt engineering techniques and when you would use each one.**
2. **Describe your process for creating prompts for different NLP tasks, such as text generation, classification, or summarization.**
3. **How do you evaluate the quality of a prompt? What metrics do you use?**
4. **Have you worked with any large language models (LLMs) before? If so, which ones and how did you use them in your prompt engineering work?**
5. **Explain how you would go about identifying and mitigating potential biases in your prompts.**

**Problem-Solving & Creativity:**

6. **Describe a time when you had to overcome a technical challenge related to prompt engineering. How did you approach the problem and what was the outcome?**
7. **Imagine you are tasked with creating a prompt for a new NLP task that has not been explored before. How would you go about brainstorming and developing a creative prompt for this task?**
8

Text Classification<br>
Classification can have many possible applications, including:<br>

Sentiment analysis<br>
Topic classification<br>
Spam detection<br>
Intent recognition<br>
Language identification<br>
Toxicity detection<br>
Emotion detection

In [9]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [10]:
prompt = """
Given a user's input, classify their intent, such as "finding information", "making a reservation", or "placing an order". \n
user input: Hi, can you please book a table for two at Juan for May 1?
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

"Making a reservation"


In [11]:
prompt = """
What is the topic for a given news headline? \n
- business \n
- entertainment \n
- health \n
- sports \n
- technology \n\n

Text: Pixel 7 Pro Expert Hands On Review. \n
The answer is: technology \n

Text: Quit smoking? \n
The answer is: health \n

Text: Birdies or bogeys? Top 5 tips to hit under par \n
The answer is: sports \n

Text: Relief from local minimum-wage hike looking more remote \n
The answer is: business \n

Text: You won't guess who just arrived in Bari, Italy for the movie premiere. \n
The answer is:
"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

entertainment 



In [12]:
import pandas as pd
from sklearn.metrics import classification_report

review_data_df = pd.DataFrame(
    {
        "review": [
            "i love this product. it does have everything i am looking for!",
            "all i can say is that you will be happy after buying this product",
            "its way too expensive and not worth the price",
            "i am feeling okay. its neither good nor too bad.",
        ],
        "sentiment_groundtruth": ["positive", "positive", "negative", "neutral"],
    }
)

review_data_df

,review,sentiment_groundtruth
0,i love this product. it does have everything i...,positive
1,all i can say is that you will be happy after ...,positive
2,its way too expensive and not worth the price,negative
3,i am feeling okay. its neither good nor too bad.,neutral


In [13]:
def get_sentiment(row):
    prompt = f"""Classify the sentiment of the following review as "positive", "neutral" or "negative". Return only the classification.
                review: {row}
              """
    response = model.generate_content(
        contents=prompt, generation_config=predictable_gen_config
    ).text
    return response


review_data_df["sentiment_prediction"] = review_data_df["review"].apply(get_sentiment)
review_data_df

,review,sentiment_groundtruth,sentiment_prediction
0,i love this product. it does have everything i...,positive,positive
1,all i can say is that you will be happy after ...,positive,positive
2,its way too expensive and not worth the price,negative,negative
3,i am feeling okay. its neither good nor too bad.,neutral,neutral


In [14]:
report = classification_report(
    review_data_df["sentiment_groundtruth"], review_data_df["sentiment_prediction"]
)
Markdown(report)

              precision    recall  f1-score   support

    negative       1.00      1.00      1.00         1
     neutral       1.00      1.00      1.00         1
    positive       1.00      1.00      1.00         2

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4


Text Summarization

In [15]:
predictable_gen_config = GenerationConfig(temperature=0.1, top_p=0.1)

In [16]:
prompt = """
Provide a very short summary, no more than three sentences, for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Summary:

"""

response = model.generate_content(
    prompt, generation_config=predictable_gen_config
)
print(response.text)

## Summary:

Quantum computers are sensitive to errors, which can be mitigated by using quantum error correction. This technique encodes information across multiple qubits, creating a "logical qubit" with lower error rates. By using logical qubits, quantum computers can achieve the accuracy needed for useful calculations. 



prefer your summary to be delivered as bullet points

In [17]:
prompt = """
Provide a very short summary in four bullet points for the following article:

Our quantum computers work by manipulating qubits in an orchestrated fashion that we call quantum algorithms.
The challenge is that qubits are so sensitive that even stray light can cause calculation errors — and the problem worsens as quantum computers grow.
This has significant consequences, since the best quantum algorithms that we know for running useful applications require the error rates of our qubits to be far lower than we have today.
To bridge this gap, we will need quantum error correction.
Quantum error correction protects information by encoding it across multiple physical qubits to form a "logical qubit," and is believed to be the only way to produce a large-scale quantum computer with error rates low enough for useful calculations.
Instead of computing on the individual qubits themselves, we will then compute on logical qubits. By encoding larger numbers of physical qubits on our quantum processor into one logical qubit, we hope to reduce the error rates to enable useful quantum algorithms.

Bullet points:

"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary in four bullet points:

* **Quantum computers use qubits, which are sensitive to errors.** Even small disturbances like stray light can cause calculation errors.
* **Current error rates are too high for useful applications.** The best quantum algorithms require much lower error rates than we have today.
* **Quantum error correction is the solution.** It encodes information across multiple qubits to create a "logical qubit" with lower error rates.
* **Logical qubits will enable useful quantum algorithms.** By encoding physical qubits into logical qubits, we can achieve the low error rates needed for practical applications. 


Summarization can also be done on other forms of documents, like an email thread or the transcript of a conversation:

In [18]:
prompt = """
Please generate a summary of the following conversation and at the end summarize the to-do's for the support Agent:

Customer: Hi, I'm Larry, and I received the wrong item.

Support Agent: Hi, Larry. How would you like to see this resolved?

Customer: That's alright. I want to return the item and get a refund, please.

Support Agent: Of course. I can process the refund for you now. Can I have your order number, please?

Customer: It's [ORDER NUMBER].

Support Agent: Thank you. I've processed the refund, and you will receive your money back within 14 days.

Customer: Thank you very much.

Support Agent: You're welcome, Larry. Have a good day!

Summary:
"""

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
Markdown(response.text)

## Summary of Conversation:

* **Customer:** Larry received the wrong item.
* **Desired Resolution:** Return the item and receive a refund.
* **Resolution:** Support Agent processed the refund.
* **Refund Timeline:** Customer will receive the refund within 14 days.

## To-Dos for Support Agent:

* **None.** The Support Agent has already completed the necessary actions. 


Text Extraction<br>
By unstructured text, we mean text that lacks a computer-readable structure like CSV, JSON, or YAML,<br>
even if a human can identify some structure (like in the ingredient list of a recipe).<br>
<br>
Some specific types of extraction include:<br>
 - Named entity recognition (NER): Extract named entities from text, including people, places, organizations, and dates.<br>
 - Relation extraction: Extract the relationships between entities in text, such as family relationships between people.<br>
 - Event extraction: Extract events from text, such as project milestones and product launches.<br>
 - Question answering: Extract information from text to answer a question.<br>

In [19]:
order = "We need eight grilled cheese sandwiches. Two with swiss cheese, three with muenster, three with cheddar."

prompt = """
    Break the customer's order into individual items with keys for the following fields:
    - item_name
    - cheese_selection

    Order:
    {order_field}
""".format(order_field=order)

response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

```json
[
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Swiss"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Muenster"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  },
  {
    "item_name": "Grilled Cheese Sandwich",
    "cheese_selection": "Cheddar"
  }
]
```


Question answering

Answering an open domain question

In [20]:
prompt = """Q: Who was President of the United States in 1955?
              Which party did he belong to?
            A:
        """
response = model.generate_content(
    contents=prompt, generation_config=predictable_gen_config
)
print(response.text)

## Who was President of the United States in 1955?

In 1955, **Dwight D. Eisenhower** was President of the United States. He was inaugurated on January 20, 1953, and served two terms, ending on January 20, 1961. 

## Which party did he belong to?

Dwight D. Eisenhower belonged to the **Republican Party**. He was the first Republican president after 20 years of Democratic presidents. 



Answering an closed domain question

In [22]:
context = """
Storage and content policy \n
How durable is my data in Cloud Storage? \n
Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
"""

question = "How is high availability achieved?"

prompt = f"""Answer the question given in the contex below:
Context: {context}?\n
Question: {question} \n
Answer:
"""

print("[Prompt]")
print(prompt)

print("[Response]")
response = model.generate_content(contents=prompt, generation_config=predictable_gen_config)
print(response.text)

[Prompt]
Answer the question given in the contex below:
Context: 
Storage and content policy 

How durable is my data in Cloud Storage? 

Cloud Storage is designed for 99.999999999% (11 9's) annual durability, which is appropriate for even primary storage and
business-critical applications. This high durability level is achieved through erasure coding that stores data pieces redundantly
across multiple devices located in multiple availability zones.
Objects written to Cloud Storage must be redundantly stored in at least two different availability zones before the
write is acknowledged as successful. Checksums are stored and regularly revalidated to proactively verify that the data
integrity of all data at rest as well as to detect corruption of data in transit. If required, corrections are automatically
made using redundant data. Customers can optionally enable object versioning to add protection against accidental deletion.
?

Question: How is high availability achieved? 

Answer:

[R